# Experiments comparing single-job LLMs to shared-job LLMs

Shared-job LLMs, LLMs where multiple customers use the same API endpoint, show considerable non-determinism (https://arxiv.org/abs/2408.04667). This experiment runs a locally deployed LLM, Llama-8b, where only a single job is run at a time. 

TL;DR single-job LLMs are deterministic so far with experiment size of 1 task/model across 10 runs of 100 college math questions. 

## Experiment setup

Penn State has an LLM on a server that we accessed with a known difficult task, college mathematics (https://huggingface.co/datasets/lukaemon/bbh) as wrapped in `tasks/college_mathematics.py`. Below are the results of running our `run_evaluation.py` code against an Azuer OpenAI endpoint `models/llama3-8b.py`. The Penn State model wrapper is not available but will be added to the repo. Once run, the `evaluate.py` script was run at Penn State and resulting `stability_eval.csv` was joined with the output from the shared-job model. 

In [5]:
import pandas as pd

df = pd.read_csv('stability_eval.csv')
display(df[['model', 'model_config', 'task_config', 'TARr', 'TARa', 'correct_count_per_run', 'num_questions', 'N', 'best_possible_accuracy', 'worst_possible_accuracy']])

,model,model_config,task_config,TARr,TARa,correct_count_per_run,num_questions,N,best_possible_accuracy,worst_possible_accuracy
0,one-job—llama3-8b,"{'temperature': 1e-14, 'seed': 12, 'top_p_k': ...","{'prompt_type': 'v2', 'shots': 0}",100.0%,96.0%,"[31, 31, 31, 31, 31, 31, 31, 31, 31, 31]",100,10,31.0%,31.0%
1,shared-job-llama3-8b,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 1.0}","{'prompt_type': 'v2', 'shots': 0}",3.0%,45.0%,"[36, 38, 32, 32, 33, 32, 35, 38, 30, 35]",100,10,55.0%,17.0%


## Experiment setup

The model have the same hyperparameters with the exception of temperature with one at `0.0` and the other at `1e-14` which is essentially zero. The `task_config` uses what we call a `v2` prompt that repaces multiple text based multiple choice prompts with the options A-D to ease parsing. 

The column `TARr` reports the percentage of times the LLM returned the exact same string result across 10 all runs, a single mismatch is sufficient to fail the criteria. `TARr` is meant to be a determinism measure for model output. 

`TARa` reports the total agreement rate of parsed answers, e.g., 'A) is the answer' is scored as the same as 'The answer is A)' without regard for whether the answer is correct. 

`correct_count_per_run` counts how many answers were correct for each run.

`num_questions` is the number of questions for the task and `N` is the number of repeat runs per questions. Tasks were executed in 1-`num_questions` order `N` times. 

`best_possible_accuracy` scores a given question correct if it was correct on any of the `N` runs, `worst_possible_accuracy` is the converse where a given question was scored wrong if any of the `N` runs was wrong. 

## Results

It is very clear that the one-job case is deterministic on outputs at a very high level of confidence given that over 1,000 measurements (100 questions x 10 runs) there were no witnessed variations across runs. We can see that the single result was with the accuracy band of the shared-job, albiet on the low end.

## Discussion

This is one task/model from our original collection of 5 models and 8 tasks where all the models were shared-job. We should run more but the preliminary results are compelling. It is premature to declare one-job models deterministic but out first experiment has demonstrated the strong possibility that this is the case. 
